# **Libraries & Packages**

In [7]:
!pip install nltk
!pip install xlsxwriter
!pip install newspaper3k
!pip install xlwt
!pip install xlrd
!pip install xlutils
!pip install gdown
!pip install -q xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
import numpy as np
from openpyxl import load_workbook
import gdown

import os
from openpyxl import load_workbook
from google.colab import files
#for token
import xlwt
import xlrd
from xlutils.copy import copy
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
#for steming
from nltk.stem import PorterStemmer
#for Lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.corpus import opinion_lexicon
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('opinion_lexicon') #Download Opinion Dictionary
import re
import warnings
from typing import Iterator, List, Tuple
from nltk.tokenize.api import TokenizerI
from nltk.tokenize.util import align_tokens
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import TreebankWordTokenizer


import xlrd
from newspaper import Article
import pandas as pd
import requests as rq
from io import BytesIO
pd.set_option('display.max_columns', None)
pd.options.display.max_columns = None

# **Getting the Files**

In [10]:
#downloading our .txt files from drive:::::::

positive_url='https://drive.google.com/file/d/11CoVQuCWjwcfwYxulDUT0oLGbDIZy3UN/view?usp=sharing'
positive_output = "positive-words.txt"

negative_url='https://drive.google.com/file/d/1SPxuq5hdHsKt9_mwjxefSmb04fzuekzR/view?usp=sharing'
negative_output = "negative-words.txt"

stop_url='https://drive.google.com/file/d/1gMOPG9gIfhUEuyRTEEoy-BdiXNSxVqTQ/view?usp=sharing'
stop_output = "StopWords_Generic.txt"

gdown.download(url = positive_url, output = positive_output, quiet=False, fuzzy=True)
gdown.download(url = negative_url, output = negative_output, quiet=False, fuzzy=True)
gdown.download(url = stop_url, output = stop_output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=11CoVQuCWjwcfwYxulDUT0oLGbDIZy3UN
To: /content/positive-words.txt
100%|██████████| 19.1k/19.1k [00:00<00:00, 8.28MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SPxuq5hdHsKt9_mwjxefSmb04fzuekzR
To: /content/negative-words.txt
100%|██████████| 44.8k/44.8k [00:00<00:00, 37.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gMOPG9gIfhUEuyRTEEoy-BdiXNSxVqTQ
To: /content/StopWords_Generic.txt
100%|██████████| 722/722 [00:00<00:00, 400kB/s]


'StopWords_Generic.txt'

# **Creating DF from EXCEL format**

In [87]:
# converting output file to dataframe
df = pd.read_excel('Output Data Structure.xlsx')

# **Main Funution for all the calculations**

In [88]:
#this fun() will run for every link availabe in excel file
path_l = []
def fun(url, ijk):
  global path_l
  

#for web scrapping:::::::::::
  article = Article(url)
  article.download()
  article.parse()
  paragraph = article.text
  sentences = nltk.sent_tokenize(paragraph)
  sentences[:0] = [article.title+'.']
  sentences.pop()

  main_para = ' '.join(map(str, sentences))


  ex = '.txt'
  filename = str((ijk+1))+ex
  f = open(filename, "w")
  f.write(main_para)
  f.close()

  pwd = os.getcwd()   
  filepath = pwd + '/'+filename
  path_l.append(filepath)
  
#importing postive words and convert into LIST:::::::::::
  my_file = open("positive-words.txt", "r")  
  data = my_file.read()
  pos_list = data.split("\n")
  my_file.close()

#importing negetive words and convert into LIST:::::::::::  
  my_file = open("negative-words.txt", "r",  encoding = "ISO-8859-1")  
  data = my_file.read()
  neg_list = data.split("\n")
  my_file.close()

#importing stop words and convert into LIST and make every words into lower_case:::::::::::
  my_file = open("StopWords_Generic.txt", "r")  
  data = my_file.read()
  custom_stop = data.split("\n")
  my_file.close()
  for i in range(len(custom_stop)):
    custom_stop[i] = custom_stop[i].lower()


#calacukting +ve, -ve and polar score:::::::::::
  def sentiment(sentence):
    pos = 0
    neg = 0
    polar = 0
    sub = 0
    words = [word.lower() for word in nltk.word_tokenize(main_para)]
  
    for word in words:
      if word in pos_list:
        pos += 1
      elif word in neg_list:
        neg += 1
  
    polar = (pos - neg)/ ((pos + neg) + 0.000001)
   
    return [pos, neg, polar]

  score = sentiment(main_para)

  pos_score = score[0]
  neg_score = score[1]
  pol_score = score[2]

#removing custom stop words and further more cleaning:::::::::::
  ps = PorterStemmer()
  lemmatizer = WordNetLemmatizer()
  sentences = nltk.sent_tokenize(main_para)
  corpus = []
  for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(custom_stop)]
    review = ' '.join(review)
    corpus.append(review)  
  

#words count after cleaning:::::::::::
  words = 0
  for i in np.arange(len(corpus)):
    words = words + len(nltk.word_tokenize(corpus[i]))  
  words_count_after_cleaning = words


#subjectivity score:::::::::::::::::
  sub = (score[0] + score[1])/ ((words) + 0.000001)

#avg sentence length::::::::::::
  sentences = nltk.sent_tokenize(main_para)
  words = nltk.word_tokenize(main_para)
  agv_sent_len = len(words) / len(sentences)


#Average Word Length::::::::::::::::::::::::::
  sentences = nltk.sent_tokenize(main_para)
  corpus = []
  for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review = review.lower()  
    review = review.split()  
    review = ' '.join(review)
    corpus.append(review)  

  m2 = ' '
  m2 = m2.join(corpus)
  lst=[x for x in m2.replace(" ", "")]
  total_char_len = len(lst)
  total_no_words = nltk.word_tokenize(m2)
  total_no_words = len(total_no_words)

  res = total_char_len / total_no_words

  


#complex_words, syllable, complex_word%age::::::::::::::::
  complexWord = 0
  syllable = 0
  complex_word_percentage = 0

  for word in words:
    vk = 0
    if word.endswith(('es','ed')):
      pass
    else:
      for w in word:
        if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
          vk += 1
          syllable += 1
          
      if(vk > 2):
        complexWord += 1

  if len(words) != 0:
    complex_word_percentage = complexWord / len(words)
  

#fog index::::::::::::::::
  fog_in = 0.4 * (agv_sent_len + complex_word_percentage)


#Personal Pronouns::::::::::::::::::::::::::::
  pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
  pronouns = (pronounRegex.findall(main_para))
  pronouns = len(pronouns)

#avg words per sentence::::::::::::::::::::::::::::
  ag_w_p_sen = agv_sent_len

  return [pos_score, neg_score, pol_score, sub, agv_sent_len, round((complex_word_percentage*100),2), round(fog_in,2), ag_w_p_sen, complexWord, words_count_after_cleaning, syllable, pronouns, round(res,2)]       

#used round just for ease!




# **Accesing the links and calling the fun()**

In [89]:
# To access excel file a=containg links:::::::::::::::
loc = ("Input.xlsx")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)

#this for loop will go through all the links and for all the links fun() will run and all the output will save in our DATAFRAME!
for i in np.arange(sheet.nrows - 1):  
  url = sheet.cell_value(i+1, 1)
  result = fun(url, i)
  df.iloc[i, 2:15] = result

In [90]:
df.insert(2, 'Extract data File location', path_l)

In [91]:
#converting float to int for some columns
df['URL_ID'] = df['URL_ID'].astype(int, errors='ignore')
df['POSITIVE SCORE'] = df['POSITIVE SCORE'].astype(int, errors='ignore')
df['NEGATIVE SCORE'] = df['NEGATIVE SCORE'].astype(int, errors='ignore')
df['AVG SENTENCE LENGTH'] = df['AVG SENTENCE LENGTH'].astype(int, errors='ignore')
df['AVG NUMBER OF WORDS PER SENTENCE'] = df['AVG NUMBER OF WORDS PER SENTENCE'].astype(int, errors='ignore')
df['COMPLEX WORD COUNT'] = df['COMPLEX WORD COUNT'].astype(int, errors='ignore')
df['WORD COUNT'] = df['WORD COUNT'].astype(int, errors='ignore')
df['SYLLABLE PER WORD'] = df['SYLLABLE PER WORD'].astype(int, errors='ignore')
df['PERSONAL PRONOUNS'] = df['PERSONAL PRONOUNS'].astype(int, errors='ignore')
df['WORD COUNT'] = df['WORD COUNT'].astype(int, errors='ignore')


# **Exporting**

In [92]:
#exporting our final dataframe to EXCEL and download the file!
writer = pd.ExcelWriter("Data Quality Associate_Intern Test OUTPUT.xlsx", engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save() 

files.download('Data Quality Associate_Intern Test OUTPUT.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>